In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [18]:
df=pd.read_csv('car-details.csv')
df.sample(10)

,name,company,model,edition,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
731,Ford Freestyle Trend Petrol BSIV,Ford,Freestyle,Trend Petrol BSIV,2019,First,Petrol,Individual,Manual,10000,44.66,1194.0,94.68,120.00,5.0,681000
3069,Hyundai i20 Active 1.2 SX with AVN,Hyundai,i20,Active 1.2 SX with AVN,2016,Second,Petrol,Individual,Manual,120000,40.40,1197.0,81.86,114.73,5.0,530000
5502,Hyundai Santro Xing XG AT eRLX Euro II,Hyundai,Santro,Xing XG AT eRLX Euro II,2006,Second,Petrol,Individual,Automatic,100000,NaN,NaN,NaN,NaN,NaN,95000
275,Hyundai i20 Asta 1.4 CRDi,Hyundai,i20,Asta 1.4 CRDi,2015,Fourth & Above,Diesel,Individual,Manual,70000,52.97,1396.0,88.73,219.70,5.0,500000
3287,Mahindra Scorpio S2 9 Seater,Mahindra,Scorpio,S2 9 Seater,2015,First,Diesel,Individual,Manual,80000,36.20,2523.0,75.00,200.00,9.0,720000
890,Hyundai Creta 1.4 EX Diesel,Hyundai,Creta,1.4 EX Diesel,2019,First,Diesel,Individual,Manual,15000,51.94,1396.0,88.70,219.70,5.0,1000000
3321,Hyundai EON D Lite Plus,Hyundai,EON,D Lite Plus,2017,First,Petrol,Individual,Manual,50000,49.60,814.0,55.20,74.50,5.0,270000
1175,Hyundai i10 Sportz,Hyundai,i10,Sportz,2009,First,Petrol,Individual,Manual,100000,47.84,1197.0,78.90,111.80,5.0,170000
3560,Ford EcoSport 1.5 TDCi Trend Plus BSIV,Ford,EcoSport,1.5 TDCi Trend Plus BSIV,2017,First,Diesel,Individual,Manual,60000,53.50,1498.0,98.59,205.00,5.0,700000
4925,Maruti Swift Dzire ZDI,Maruti,Swift,Dzire ZDI,2013,First,Diesel,Individual,Manual,149000,55.00,1248.0,74.00,190.00,5.0,475000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [20]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [21]:
df.shape

(6926, 16)

In [22]:
for col in df.select_dtypes(include='O').columns:
    print(f'column:{col}')
    print(f'cardinality:{df[col].nunique()}')
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()

column:name
cardinality:2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                          0.017037
Maruti Alto 800 LXI                             0.010973
Maruti Alto LXi                                 0.009962
Maruti Swift VDI                                0.008663
Maruti Swift VDI BSIV                           0.008085
                                                  ...   
Ford EcoSport 1.5 TDCi Titanium Plus BE BSIV    0.000144
Ford Figo 1.5 Sports Edition MT                 0.000144
Volkswagen Polo Select 1.2 MPI Highline         0.000144
Honda Jazz V Diesel                             0.000144
Renault Duster RXZ 110PS AMT BSIV               0.000144
Name: proportion, Length: 2058, dtype: float64

column:company
cardinality:32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' 'Ford

In [23]:
df=df.drop(columns=['name','model','edition'])
df.head(10)

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00000,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70000,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60000,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06000,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84000,1298.0,88.20,112.776475,5.0,130000
5,Hyundai,2017,First,Petrol,Individual,Manual,45000,47.34000,1197.0,81.86,113.750000,5.0,440000
6,Maruti,2007,First,LPG,Individual,Manual,175000,0.02768,1061.0,57.50,76.491870,5.0,96000
7,Maruti,2001,Second,Petrol,Individual,Manual,5000,37.84000,796.0,37.00,59.000000,4.0,45000
8,Toyota,2011,First,Diesel,Individual,Manual,90000,55.44000,1364.0,67.10,170.000000,5.0,350000
9,Ford,2013,First,Diesel,Individual,Manual,169000,47.00000,1399.0,68.10,160.000000,5.0,200000


In [24]:
df=df.drop_duplicates()
df.head(10)

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00000,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70000,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60000,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06000,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84000,1298.0,88.20,112.776475,5.0,130000
5,Hyundai,2017,First,Petrol,Individual,Manual,45000,47.34000,1197.0,81.86,113.750000,5.0,440000
6,Maruti,2007,First,LPG,Individual,Manual,175000,0.02768,1061.0,57.50,76.491870,5.0,96000
7,Maruti,2001,Second,Petrol,Individual,Manual,5000,37.84000,796.0,37.00,59.000000,4.0,45000
8,Toyota,2011,First,Diesel,Individual,Manual,90000,55.44000,1364.0,67.10,170.000000,5.0,350000
9,Ford,2013,First,Diesel,Individual,Manual,169000,47.00000,1399.0,68.10,160.000000,5.0,200000


In [25]:
df.duplicated().sum()

np.int64(0)

In [26]:
X=df.drop(columns=['selling_price'])
y=df.selling_price.copy()
print(X.shape,y.shape)

(6907, 12) (6907,)


In [27]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)


(5525, 12) (5525,)
(1382, 12) (1382,)


In [28]:
num_cols=X_train.select_dtypes(include='number').columns.tolist()
cat_cols=[col for col in X_train.columns if col not in num_cols]

print(num_cols)
print(cat_cols)

['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats']
['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [31]:
num_pipe=Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                         ('scaler',StandardScaler())
                         ])

cat_pipe=Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                          ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
                     ])

preprocessor=ColumnTransformer(transformers=[('num', num_pipe, num_cols), ('cat',cat_pipe, cat_cols)])

regressor=RandomForestRegressor(
    n_estimators=10,
    max_depth=5,
    random_state=42

)

rf_model=Pipeline(steps=[('pre',preprocessor),
                         ('reg',regressor)
                         ])


rf_model.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('pre', ...), ('reg', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains spar

In [46]:
y_train_pred=rf_model.predict(X_train)
train_mse=mean_squared_error(y_train,y_train_pred)
print(f'Train RMSE:{train_mse:,.3f}')


y_test_pred=rf_model.predict(X_test)
test_mse=mean_squared_error(y_test,y_test_pred)
print(f'Test RMSE:{test_mse:,.3f}')


print ('###################################################')



y_train_pred=rf_model.predict(X_train)
train_rmse=root_mean_squared_error(y_train,y_train_pred,)
print(f'Train RMSE:{train_rmse:,.3f}')


y_test_pred=rf_model.predict(X_test)
test_rmse=root_mean_squared_error(y_test,y_test_pred)
print(f'Test RMSE:{test_rmse:,.3f}')

Train RMSE:28,882,149,235.109
Test RMSE:29,719,046,955.023
###################################################
Train RMSE:169,947.490
Test RMSE:172,392.131


In [44]:
import sklearn
print(sklearn.__version__)

1.8.0
